In [13]:
import pickle
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import chi2
import numpy as np

In [14]:

data = pd.read_csv('C:\\Users\\baril\\OneDrive\\Desktop\\3395Compe\\train.csv', sep = ',', header=0).to_numpy()
data = pd.DataFrame(data = data[:,1:3], columns = ["Article", "Categorie"])
data.head()


,Article,Categorie
0,The energy released in a solar flare is part...,astro-ph
1,In light of current atmospheric neutrino osc...,hep-ph
2,We consider the following basic learning tas...,cs.LG
3,"In this paper, we characterise the family of...",math.CO
4,The control of condensed matter systems out ...,cond-mat.mes-hall


In [15]:

data['Article_mod'] = data['Article'].str.replace("\r", " ")
data['Article_mod'] = data['Article_mod'].str.replace("\n", " ")
data['Article_mod'] = data['Article_mod'].str.replace("    ", " ")
data['Article_mod'] = data['Article_mod'].str.replace('"', '')
data['Article_mod'] = data['Article_mod'].str.lower()
punctuation_signs = list("?:!.,;")

for punct_sign in punctuation_signs:
    data['Article_mod'] = data['Article_mod'].str.replace(punct_sign, '')
data['Article_mod'] = data['Article_mod'].str.replace("'s", "")


In [16]:
nltk.download('punkt')
nltk.download('wordnet')

lemmatizer = WordNetLemmatizer()
n_obs = len(data)
word_bag_list = [] #tous les sacs du DF

for ligne in range(0, n_obs):
    word_bag = [] #sac de mot par article
    
    texte= data.loc[ligne]['Article_mod']
    mots = texte.split(" ")

    for mot in mots:
        word_bag.append(lemmatizer.lemmatize(mot, pos="v"))
        
    text_split = " ".join(word_bag)
    
    word_bag_list.append(text_split)

data['Article_mod'] = word_bag_list

[nltk_data] Downloading package punkt to C:\Users\baril/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to C:\Users\baril/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [17]:
nltk.download('stopwords')

no_word = list(stopwords.words('english'))

for mot in no_word:
    reg = r"\b" + mot + r"\b"
    data['Article_mod'] = data['Article_mod'].str.replace(reg, '')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\baril/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [18]:
list_columns = ["Categorie", "Article", "Article_mod"]
data = data[list_columns]
np.unique(data.Categorie)
categorie_codes = {
    'astro-ph': 0,
    'astro-ph.CO': 1,
    'astro-ph.GA': 2,
    'astro-ph.SR': 3,
    'cond-mat.mes-hall': 4,
    'cond-mat.mtrl-sci': 5,
    'cs.LG': 6,
    'gr-qc': 7,
    'hep-ph' : 8,
    'hep-th' : 9,
    'math.AP' : 10,
    'math.CO' : 11,
    'physics.optics' : 12,
    'quant-ph' : 13,
    'stat.ML' : 14
}
data['Categorie_Codes'] = data['Categorie']
data = data.replace({'Categorie_Code':categorie_codes})
data.head()

,Categorie,Article,Article_mod,Categorie_Codes
0,astro-ph,The energy released in a solar flare is part...,energy release solar flare partition th...,astro-ph
1,hep-ph,In light of current atmospheric neutrino osc...,light current atmospheric neutrino oscilla...,hep-ph
2,cs.LG,We consider the following basic learning tas...,consider follow basic learn task give inde...,cs.LG
3,math.CO,"In this paper, we characterise the family of...",paper characterise family finite arc-tr...,math.CO
4,cond-mat.mes-hall,The control of condensed matter systems out ...,control condense matter systems equilibr...,cond-mat.mes-hall


In [23]:
train_x, test_x, train_y, test_y = train_test_split(data['Article_mod'], 
                                                   data['Categorie_Codes'],
                                                   test_size=0.2,
                                                   random_state=11)  
#TFIDF
ngram_range = (1,2)
min_df = 10
max_df = 1.
max_features = 3000

tfidf = TfidfVectorizer(encoding='utf-8',
                        ngram_range=(1,2),
                        stop_words=None,
                        lowercase=False,
                        max_df=1.,
                        min_df=10,
                        max_features=3000, ##à jouer avec
                        norm='l2',
                        sublinear_tf=True)
                        
train_x = tfidf.fit_transform(train_x).toarray()
test_x = tfidf.transform(test_x).toarray()


In [ ]:
from sklearn import svm
np.random.seed(42)

SVM = svm.SVC(kernel='linear')
SVM.fit(train_x,train_y)
predictions_SVM = SVM.predict(test_x)
print("SVM Accuracy Score (Validation) -> ",accuracy_score(predictions_SVM, test_y)*100)
predictions_SVM = SVM.predict(train_x)
print("SVM Accuracy Score (Train) -> ",accuracy_score(predictions_SVM, train_y)*100)
